In [2]:
# Importing the necessary packages
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import StandardScaler

import warnings
warnings.simplefilter(action='ignore')

In [3]:
# Load the dataset
dataset = pd.read_csv('D:/Chools/Day_10/diabetes.csv')

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [5]:
# Split features and target variable
X = dataset.drop('Outcome', axis=1)
y = dataset['Outcome']

In [6]:
# Standardization
a = StandardScaler()
a.fit(X)
X_standardized = a.transform(X)

In [7]:
pd.DataFrame(X_standardized).describe()

,0,1,2,3,4,5,6,7
count,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02
mean,-6.476301e-17,-9.251859e-18,1.503427e-17,1.006140e-16,-3.006854e-17,2.590520e-16,2.451743e-16,1.931325e-16
std,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00
min,-1.141852e+00,-3.783654e+00,-3.572597e+00,-1.288212e+00,-6.928906e-01,-4.060474e+00,-1.189553e+00,-1.041549e+00
25%,-8.448851e-01,-6.852363e-01,-3.673367e-01,-1.288212e+00,-6.928906e-01,-5.955785e-01,-6.889685e-01,-7.862862e-01
50%,-2.509521e-01,-1.218877e-01,1.496408e-01,1.545332e-01,-4.280622e-01,9.419788e-04,-3.001282e-01,-3.608474e-01
75%,6.399473e-01,6.057709e-01,5.632228e-01,7.190857e-01,4.120079e-01,5.847705e-01,4.662269e-01,6.602056e-01
max,3.906578e+00,2.444478e+00,2.734528e+00,4.921866e+00,6.652839e+00,4.455807e+00,5.883565e+00,4.063716e+00


#### Tuning of Hyperparameters :- Batch Size and Epochs

In [24]:
# Importing the necessary packages
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
# from keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier
from keras.optimizers import Adam

In [25]:
# Define the create_model function
def create_model(learning_rate=0.01):
    model = Sequential()
    model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    
    adam = Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model


In [26]:
# Create the model
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid search parameters
batch_size = [10, 20, 40]
epochs = [10, 50, 100]
learning_rate = [0.01, 0.001]  # Add learning rate as a parameter to be tuned

# Make a dictionary of the grid search parameters
param_grid = dict(batch_size=batch_size, epochs=epochs)

# Build and fit the GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=KFold(), verbose=10)
grid_result = grid.fit(X_standardized, y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START batch_size=10, epochs=10....................................
[CV 1/5; 1/9] END .....batch_size=10, epochs=10;, score=0.747 total time=   0.8s
[CV 2/5; 1/9] START batch_size=10, epochs=10....................................
[CV 2/5; 1/9] END .....batch_size=10, epochs=10;, score=0.701 total time=   0.8s
[CV 3/5; 1/9] START batch_size=10, epochs=10....................................
[CV 3/5; 1/9] END .....batch_size=10, epochs=10;, score=0.766 total time=   0.8s
[CV 4/5; 1/9] START batch_size=10, epochs=10....................................
[CV 4/5; 1/9] END .....batch_size=10, epochs=10;, score=0.843 total time=   0.8s
[CV 5/5; 1/9] START batch_size=10, epochs=10....................................
[CV 5/5; 1/9] END .....batch_size=10, epochs=10;, score=0.778 total time=   0.8s
[CV 1/5; 2/9] START batch_size=10, epochs=50....................................
[CV 1/5; 2/9] END .....batch_size=10, epochs=50;,

In [27]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.7735166793990322, using {'batch_size': 40, 'epochs': 10}
0.7670401493930906,0.04613071167935996 with: {'batch_size': 10, 'epochs': 10}
0.7488158899923606,0.0460453591743617 with: {'batch_size': 10, 'epochs': 50}
0.7513793396146337,0.033958958814046536 with: {'batch_size': 10, 'epochs': 100}
0.7631355572532044,0.04917230904824848 with: {'batch_size': 20, 'epochs': 10}
0.7539767422120363,0.03475760566580164 with: {'batch_size': 20, 'epochs': 50}
0.7487479840421016,0.02940797353592768 with: {'batch_size': 20, 'epochs': 100}
0.7735166793990322,0.036735767771314834 with: {'batch_size': 40, 'epochs': 10}
0.753993718699601,0.03562738075215721 with: {'batch_size': 40, 'epochs': 50}
0.750097614803497,0.03560195909720659 with: {'batch_size': 40, 'epochs': 100}


#### Tuning of Hyperparameters:- Learning rate and Drop out rate

In [11]:
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, KFold

# Defining the model
def create_model(learning_rate, dropout_rate, activation_function, kernel_initializer):
    model = Sequential()
    model.add(Dense(8, input_dim=8, kernel_initializer=kernel_initializer, activation=activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, kernel_initializer=kernel_initializer, activation=activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    adam = Adam(lr=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

# Create the model with dropout_rate parameter in KerasClassifier constructor
model = KerasClassifier(build_fn=create_model, verbose=0, batch_size=40, epochs=10, dropout_rate=0.0)

# Define the grid search parameters
learning_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2]
activation_function = ['relu', 'sigmoid', 'tanh', 'linear']  # Add activation function parameter
kernel_initializer = ['glorot_uniform', 'normal', 'zero']  # Add kernel initializer parameter

# Make a dictionary of the grid search parameters
param_grids = dict(learning_rate=learning_rate, dropout_rate=dropout_rate, 
                   activation_function=activation_function, kernel_initializer=kernel_initializer)

# Build and fit the GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grids, cv=KFold(), verbose=10)
grid_result = grid.fit(X_standardized, y)


Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV 1/5; 1/108] START activation_function=relu, dropout_rate=0.0, kernel_initializer=glorot_uniform, learning_rate=0.001
[CV 1/5; 1/108] END activation_function=relu, dropout_rate=0.0, kernel_initializer=glorot_uniform, learning_rate=0.001;, score=0.649 total time=   0.6s
[CV 2/5; 1/108] START activation_function=relu, dropout_rate=0.0, kernel_initializer=glorot_uniform, learning_rate=0.001
[CV 2/5; 1/108] END activation_function=relu, dropout_rate=0.0, kernel_initializer=glorot_uniform, learning_rate=0.001;, score=0.675 total time=   0.5s
[CV 3/5; 1/108] START activation_function=relu, dropout_rate=0.0, kernel_initializer=glorot_uniform, learning_rate=0.001
[CV 3/5; 1/108] END activation_function=relu, dropout_rate=0.0, kernel_initializer=glorot_uniform, learning_rate=0.001;, score=0.727 total time=   0.5s
[CV 4/5; 1/108] START activation_function=relu, dropout_rate=0.0, kernel_initializer=glorot_uniform, learning_rate=0.0

In [12]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.7761225700378418, using {'activation_function': 'linear', 'dropout_rate': 0.2, 'kernel_initializer': 'normal', 'learning_rate': 0.01}
0.6835922360420227,0.04317951184336888 with: {'activation_function': 'relu', 'dropout_rate': 0.0, 'kernel_initializer': 'glorot_uniform', 'learning_rate': 0.001}
0.7656905174255371,0.038743037620730746 with: {'activation_function': 'relu', 'dropout_rate': 0.0, 'kernel_initializer': 'glorot_uniform', 'learning_rate': 0.01}
0.7513963222503662,0.0474620854735074 with: {'activation_function': 'relu', 'dropout_rate': 0.0, 'kernel_initializer': 'glorot_uniform', 'learning_rate': 0.1}
0.7526186347007752,0.017231110570084692 with: {'activation_function': 'relu', 'dropout_rate': 0.0, 'kernel_initializer': 'normal', 'learning_rate': 0.001}
0.7735421419143677,0.04413110172855617 with: {'activation_function': 'relu', 'dropout_rate': 0.0, 'kernel_initializer': 'normal', 'learning_rate': 0.01}
0.7422629714012146,0.04239020086519357 with: {'activation_function

#### Tuning of Hyperparameters:- Activation Function and Kernel Initializer

In [14]:
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, KFold

# Defining the model
def create_model(activation_function, init, dropout_rate=0.1):
    model = Sequential()
    model.add(Dense(8, input_dim=8, kernel_initializer=init, activation=activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, input_dim=8, kernel_initializer=init, activation=activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    adam = Adam(lr=0.001)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

# Create the model
model = KerasClassifier(build_fn=create_model, verbose=0, batch_size=40, epochs=10)

# Define the grid search parameters
activation_function = ['softmax', 'relu', 'tanh', 'linear']
init = ['uniform', 'normal', 'zero']
dropout_rate = [0.0, 0.1, 0.2]  # Add dropout rate parameter

# Make a dictionary of the grid search parameters
param_grids = dict(activation_function=activation_function, init=init, dropout_rate=dropout_rate)

# Build and fit the GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grids, cv=KFold(), verbose=10)
grid_result = grid.fit(X_standardized, y)


Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5; 1/36] START activation_function=softmax, dropout_rate=0.0, init=uniform
[CV 1/5; 1/36] END activation_function=softmax, dropout_rate=0.0, init=uniform;, score=0.649 total time=   0.7s
[CV 2/5; 1/36] START activation_function=softmax, dropout_rate=0.0, init=uniform
[CV 2/5; 1/36] END activation_function=softmax, dropout_rate=0.0, init=uniform;, score=0.584 total time=   0.5s
[CV 3/5; 1/36] START activation_function=softmax, dropout_rate=0.0, init=uniform
[CV 3/5; 1/36] END activation_function=softmax, dropout_rate=0.0, init=uniform;, score=0.630 total time=   0.5s
[CV 4/5; 1/36] START activation_function=softmax, dropout_rate=0.0, init=uniform
[CV 4/5; 1/36] END activation_function=softmax, dropout_rate=0.0, init=uniform;, score=0.745 total time=   0.5s
[CV 5/5; 1/36] START activation_function=softmax, dropout_rate=0.0, init=uniform
[CV 5/5; 1/36] END activation_function=softmax, dropout_rate=0.0, init=uniform;, scor

In [15]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.7605296730995178, using {'activation_function': 'tanh', 'dropout_rate': 0.2, 'init': 'normal'}
0.6511586427688598,0.05244526932680711 with: {'activation_function': 'softmax', 'dropout_rate': 0.0, 'init': 'uniform'}
0.5011713802814484,0.15999395771379943 with: {'activation_function': 'softmax', 'dropout_rate': 0.0, 'init': 'normal'}
0.5914183855056763,0.1313092546244197 with: {'activation_function': 'softmax', 'dropout_rate': 0.0, 'init': 'zero'}
0.5531194269657135,0.15092304368625437 with: {'activation_function': 'softmax', 'dropout_rate': 0.1, 'init': 'uniform'}
0.6511586427688598,0.05244526932680711 with: {'activation_function': 'softmax', 'dropout_rate': 0.1, 'init': 'normal'}
0.6173924148082733,0.10871275688841411 with: {'activation_function': 'softmax', 'dropout_rate': 0.1, 'init': 'zero'}
0.6511586427688598,0.05244526932680711 with: {'activation_function': 'softmax', 'dropout_rate': 0.2, 'init': 'uniform'}
0.6511586427688598,0.05244526932680711 with: {'activation_functio

#### Tuning of Hyperparameter :-Number of Neurons in activation layer

In [17]:
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, KFold

# Defining the model
def create_model(neuron1, neuron2, dropout_rate=0.1):
    model = Sequential()
    model.add(Dense(neuron1, input_dim=8, kernel_initializer='uniform', activation='tanh'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neuron2, input_dim=neuron1, kernel_initializer='uniform', activation='tanh'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    adam = Adam(lr=0.001)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

# Create the model
model = KerasClassifier(build_fn=create_model, verbose=0, batch_size=40, epochs=10)

# Define the grid search parameters
neuron1 = [4, 8, 16]
neuron2 = [2, 4, 8]
dropout_rate = [0.0, 0.1, 0.2]  # Add dropout rate parameter

# Make a dictionary of the grid search parameters
param_grids = dict(neuron1=neuron1, neuron2=neuron2, dropout_rate=dropout_rate)

# Build and fit the GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grids, cv=KFold(), verbose=10)
grid_result = grid.fit(X_standardized, y)


Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5; 1/27] START dropout_rate=0.0, neuron1=4, neuron2=2.....................
[CV 1/5; 1/27] END dropout_rate=0.0, neuron1=4, neuron2=2;, score=0.747 total time=   0.5s
[CV 2/5; 1/27] START dropout_rate=0.0, neuron1=4, neuron2=2.....................
[CV 2/5; 1/27] END dropout_rate=0.0, neuron1=4, neuron2=2;, score=0.688 total time=   0.5s
[CV 3/5; 1/27] START dropout_rate=0.0, neuron1=4, neuron2=2.....................
[CV 3/5; 1/27] END dropout_rate=0.0, neuron1=4, neuron2=2;, score=0.708 total time=   1.0s
[CV 4/5; 1/27] START dropout_rate=0.0, neuron1=4, neuron2=2.....................
[CV 4/5; 1/27] END dropout_rate=0.0, neuron1=4, neuron2=2;, score=0.804 total time=   0.5s
[CV 5/5; 1/27] START dropout_rate=0.0, neuron1=4, neuron2=2.....................
[CV 5/5; 1/27] END dropout_rate=0.0, neuron1=4, neuron2=2;, score=0.732 total time=   0.5s
[CV 1/5; 2/27] START dropout_rate=0.0, neuron1=4, neuron2=4...................

In [18]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.7696290612220764, using {'dropout_rate': 0.0, 'neuron1': 16, 'neuron2': 2}
0.7357609868049622,0.039533107185542926 with: {'dropout_rate': 0.0, 'neuron1': 4, 'neuron2': 2}
0.7435701608657836,0.041171628899017466 with: {'dropout_rate': 0.0, 'neuron1': 4, 'neuron2': 4}
0.7552839279174804,0.03873777214845699 with: {'dropout_rate': 0.0, 'neuron1': 4, 'neuron2': 8}
0.747508704662323,0.04642591760479889 with: {'dropout_rate': 0.0, 'neuron1': 8, 'neuron2': 2}
0.7553009033203125,0.04316958224239693 with: {'dropout_rate': 0.0, 'neuron1': 8, 'neuron2': 4}
0.7656990051269531,0.03468605788127298 with: {'dropout_rate': 0.0, 'neuron1': 8, 'neuron2': 8}
0.7696290612220764,0.04160158455691619 with: {'dropout_rate': 0.0, 'neuron1': 16, 'neuron2': 2}
0.7696120858192443,0.0404945687156337 with: {'dropout_rate': 0.0, 'neuron1': 16, 'neuron2': 4}
0.7682964086532593,0.035408110721954086 with: {'dropout_rate': 0.0, 'neuron1': 16, 'neuron2': 8}
0.7422375082969666,0.032185949878950064 with: {'dropout_r

#### Training model with optimum values of Hyperparameters

In [28]:
from sklearn.metrics import accuracy_score

# Define the model with the optimum parameters
def create_model():
    model = Sequential()
    model.add(Dense(16, input_dim=8, kernel_initializer='normal', activation='linear'))
    model.add(Dropout(0.2))
    model.add(Dense(4, kernel_initializer='normal', activation='linear'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    
    adam = Adam(lr=0.01)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

# Create the model
model = KerasClassifier(build_fn=create_model, verbose=0, batch_size=40, epochs=10)

# Fit the model
model.fit(X_standardized, y)

# Predict using the trained model
y_predict = model.predict(X_standardized)

# Print the accuracy score
print("Accuracy Score:", accuracy_score(y, y_predict))

Accuracy Score: 0.77734375


# Hyperparameters all at once


The hyperparameter optimization was carried out by taking 2 hyperparameters at once. We may have missed the best values. The performance can be further improved by finding the optimum values of hyperparameters all at once given by the code snippet below.
#### This process is computationally expensive.

In [22]:
def create_model(learning_rate,dropout_rate,activation_function,init,neuron1,neuron2):
    model = Sequential()
    model.add(Dense(neuron1,input_dim = 8,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neuron2,input_dim = neuron1,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(lr = learning_rate)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0)

# Define the grid search parameters

batch_size = [10,20,40]
epochs = [10,50,100]
learning_rate = [0.001,0.01,0.1]
dropout_rate = [0.0,0.1,0.2]
activation_function = ['softmax','relu','tanh','linear']
init = ['uniform','normal','zero']
neuron1 = [4,8,16]
neuron2 = [2,4,8]

# Make a dictionary of the grid search parameters

param_grids = dict(batch_size = batch_size,epochs = epochs,learning_rate = learning_rate,dropout_rate = dropout_rate,
                   activation_function = activation_function,init = init,neuron1 = neuron1,neuron2 = neuron2)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_standardized,y)

# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Fitting 5 folds for each of 8748 candidates, totalling 43740 fits
[CV 1/5; 1/8748] START activation_function=softmax, batch_size=10, dropout_rate=0.0, epochs=10, init=uniform, learning_rate=0.001, neuron1=4, neuron2=2
[CV 1/5; 1/8748] END activation_function=softmax, batch_size=10, dropout_rate=0.0, epochs=10, init=uniform, learning_rate=0.001, neuron1=4, neuron2=2;, score=0.649 total time=   0.9s
[CV 2/5; 1/8748] START activation_function=softmax, batch_size=10, dropout_rate=0.0, epochs=10, init=uniform, learning_rate=0.001, neuron1=4, neuron2=2
[CV 2/5; 1/8748] END activation_function=softmax, batch_size=10, dropout_rate=0.0, epochs=10, init=uniform, learning_rate=0.001, neuron1=4, neuron2=2;, score=0.584 total time=   0.9s
[CV 3/5; 1/8748] START activation_function=softmax, batch_size=10, dropout_rate=0.0, epochs=10, init=uniform, learning_rate=0.001, neuron1=4, neuron2=2
[CV 3/5; 1/8748] END activation_function=softmax, batch_size=10, dropout_rate=0.0, epochs=10, init=uniform, lear

KeyboardInterrupt: 

In [29]:
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, KFold

# Defining the model
def create_model(learning_rate, dropout_rate, activation_function, init, neuron1, neuron2):
    model = Sequential()
    model.add(Dense(neuron1, input_dim=8, kernel_initializer=init, activation=activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neuron2, input_dim=neuron1, kernel_initializer=init, activation=activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    adam = Adam(lr=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

# Create the model
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid search parameters
batch_size = [20, 40]
epochs = [50, 100]
learning_rate = [0.0001, 0.001, 0.01]
dropout_rate = [0.1, 0.2]
activation_function = ['relu', 'tanh']
init = ['uniform', 'normal']
neuron1 = [8, 16]
neuron2 = [4, 8]

# Make a dictionary of the grid search parameters
param_grids = dict(batch_size=batch_size, epochs=epochs, learning_rate=learning_rate,
                   dropout_rate=dropout_rate, activation_function=activation_function,
                   init=init, neuron1=neuron1, neuron2=neuron2)

# Build and fit the GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grids, cv=KFold(), verbose=10)
grid_result = grid.fit(X_standardized, y)

# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{},{} with: {}'.format(mean, stdev, param))


Fitting 5 folds for each of 384 candidates, totalling 1920 fits
[CV 1/5; 1/384] START activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=50, init=uniform, learning_rate=0.0001, neuron1=8, neuron2=4
[CV 1/5; 1/384] END activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=50, init=uniform, learning_rate=0.0001, neuron1=8, neuron2=4;, score=0.779 total time=   1.7s
[CV 2/5; 1/384] START activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=50, init=uniform, learning_rate=0.0001, neuron1=8, neuron2=4
[CV 2/5; 1/384] END activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=50, init=uniform, learning_rate=0.0001, neuron1=8, neuron2=4;, score=0.584 total time=   1.6s
[CV 3/5; 1/384] START activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=50, init=uniform, learning_rate=0.0001, neuron1=8, neuron2=4
[CV 3/5; 1/384] END activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=50, init=uniform, learning_rate=0.0001, neu